In [1]:
# pip install pandas
import pandas as pd
import glob
from datetime import timedelta

# Path to folder containing all participant CSVs
path = "C:\Users\jasmi\OneDrive\Skrivebord\DTU\5. semester\Eksperimenter i Kognitionsvidenskab\experiment\02455-Experiment-In-Cognitive-Science\analysis\sample_trial.csv"  # <-- change to your folder path
files = sorted(glob.glob(f"{path}/*.csv"))

participants = []

for file in files:
    df = pd.read_csv(file)
    
    # Extract participant ID from filename (assuming 'participant_01.csv' format)
    participant_id = file.split("/")[-1].split(".")[0]
    
    # Convert experiment start time to datetime
    experiment_start = pd.to_datetime(df.loc[0, "experiment_start"], errors="coerce")
    
    # Calculate absolute start/stop times
    df["video_start_time"] = experiment_start + pd.to_timedelta(df["video_start_seconds"], unit="s")
    df["video_stop_time"] = experiment_start + pd.to_timedelta(df["video_stop_seconds"], unit="s")
    
    # Compute experiment end as last video stop
    experiment_end = df["video_stop_time"].max()
    
    # Build video objects
    videos = []
    for _, row in df.iterrows():
        video_data = {
            "video_id": row.get("video_id", None),
            "video_belief": row.get("belief", None),  # AI / Human
            "video_start_time": row["video_start_time"].isoformat() if pd.notnull(row["video_start_time"]) else None,
            "video_stop_time": row["video_stop_time"].isoformat() if pd.notnull(row["video_stop_time"]) else None,
            "trusted_label": None,    # Placeholder for later
            "sam_scores": [None, None, None],  # [valence, arousal, dominance]
        }
        videos.append(video_data)
    
    participant_data = {
        "participant_id": participant_id,
        "experiment_start": experiment_start.isoformat() if pd.notnull(experiment_start) else None,
        "experiment_end": experiment_end.isoformat() if pd.notnull(experiment_end) else None,
        "videos": videos,
        # placeholders for future expansion
        "attention_check_passed": None,
        "comments": None,
    }
    
    participants.append(participant_data)

# Verify structure
print(f"Loaded {len(participants)} participants")
print(participants[0])  # Show first one

# Optional: save as JSON
import json
with open("experiment_data.json", "w") as f:
    json.dump(participants, f, indent=4)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (192656749.py, line 7)